# 1. Load and Filter Benchmark Results

- **Load benchmark results** from the CSV file `benchmark_results.csv`.
- **Filter out rows** where either `Runtime (s)` (measured runtime) or `Reported Runtime (s)` is missing. This ensures only runs with valid runtime data are considered.
- **Keep only successful solver runs** where `Status == "ok"`, ignoring failed or timed-out runs.

In [2]:
import sys

import pandas as pd

sys.path.insert(0, "..")
from runner.utils import load_benchmark_metadata

# Load benchmark results into benchmark_data
benchmark_data = pd.read_csv("../results/benchmark_results.csv")

# Keep only rows where both measured and reported runtimes exist
benchmark_data = benchmark_data[
    (~benchmark_data["Runtime (s)"].isna())
    & (~benchmark_data["Reported Runtime (s)"].isna())
]

print(f"Rows with both runtimes available: {len(benchmark_data)}")

# Keep only successful solver runs
benchmark_data = benchmark_data[benchmark_data["Status"] == "ok"]

print(f"Rows after filtering Status==ok: {len(benchmark_data)}")


Rows with both runtimes available: 2554
Rows after filtering Status==ok: 1637


# 2. Compute Runtime Differences and Add Identifier Columns

- **Compute absolute runtime difference** (`runtime-difference`) between the measured runtime (`Runtime (s)`) and the reported runtime (`Reported Runtime (s)`).
- **Compute relative runtime difference** (`runtime-difference-%`) as a percentage of the larger of the two runtimes. 
- **Construct `bench-size` column** as a unique identifier for each benchmark instance by combining the benchmark name and size.
- **Construct `solver-version` column** as a unique identifier for each solver run by combining the solver name and its version.

In [3]:
# Compute absolute and percentage runtime differences
benchmark_data["runtime-difference"] = (
    benchmark_data["Runtime (s)"]
    - benchmark_data["Reported Runtime (s)"]
).abs()

benchmark_data["runtime-difference-%"] = (
    benchmark_data["runtime-difference"]
    / benchmark_data[["Runtime (s)", "Reported Runtime (s)"]].max(axis=1)
) * 100

# Construct bench-size and solver-version in benchmark_data
benchmark_data["bench-size"] = (
    benchmark_data["Benchmark"] + "-" + benchmark_data["Size"]
)

benchmark_data["solver-version"] = (
    benchmark_data["Solver"] + "-" + benchmark_data["Solver Version"]
)

# Keep only relevant columns for further analysis
cols_to_export = [
    "bench-size",
    "solver-version",
    "Runtime (s)",
    "Reported Runtime (s)",
    "runtime-difference",
    "runtime-difference-%",
]

benchmark_data = benchmark_data[cols_to_export]

# Display the first few rows of `benchmark_data`
display(benchmark_data.head())


,bench-size,solver-version,Runtime (s),Reported Runtime (s),runtime-difference,runtime-difference-%
0,TIMES-GEO-global-netzero-31-20ts,gurobi-13.0.0,30621.769838,30590.708018,31.061819,0.101437
3,TIMES-GEO-global-netzero-31-20ts,highs-1.12.0,62869.744979,62820.879014,48.865966,0.077726
6,pypsa-de-elec-50-1h,gurobi-13.0.0,2989.140083,2923.996410,65.143673,2.179345
12,temoa-US_9R_TS-9-12ts,gurobi-13.0.0,190.656273,172.031106,18.625167,9.768977
13,temoa-US_9R_TS-9-12ts,highs-hipo-1.12.0-hipo,3181.154351,3181.154351,0.000000,0.000000


# 3. Load and Prepare Benchmark Metadata

- Load metadata from `metadata.yaml` using `load_benchmark_metadata()`. This contains details about each benchmark instance, such as number of variables, number of constraints, and size category.

- Construct `bench-size` column as a unique identifier for each benchmark instance by combining `Benchmark` and `Instance`. This will be used to merge metadata with runtime results.

- Copy `Size` column to a new column called `bench-size-category` to keep the size category information (S, M, L, etc.) separate.

In [4]:
# Load benchmark metadata
benchmark_metadata = load_benchmark_metadata("../results/metadata.yaml")

# Construct bench-size in benchmark_metadata
benchmark_metadata["bench-size"] = (
    benchmark_metadata["Benchmark"]
    + "-"
    + benchmark_metadata["Instance"]
)

# Copy Size column to a new column called "bench-size-category"
benchmark_metadata["bench-size-category"] = benchmark_metadata["Size"]

# Keep only relevant metadata fields
meta_cols = [
    "bench-size",
    "Num. variables",
    "Num. constraints",
    "bench-size-category",
]

benchmark_metadata = benchmark_metadata[meta_cols]

# Display the first few rows of `benchmark_metadata`
display(benchmark_metadata.head())


,bench-size,Num. variables,Num. constraints,bench-size-category
ethos_fine_europe_60tp-175-720ts,ethos_fine_europe_60tp-175-720ts,8141182,9230938,L
ethos-fine-multi-regional-7tp-8-168ts,ethos-fine-multi-regional-7tp-8-168ts,55979,91178,M
ethos-fine-multi-regional-7tp-12seg-8-84ts,ethos-fine-multi-regional-7tp-12seg-8-84ts,33215,64214,M
ethos-fine-energyland-full-timeseries-1-8760ts,ethos-fine-energyland-full-timeseries-1-8760ts,473179,543229,M
ethos-fine-energyland-48tp-1-1152ts,ethos-fine-energyland-48tp-1-1152ts,32770,40198,M


# 3. Merge Benchmark Results with Metadata and Export

- **Merge `benchmark_data` with `benchmark_metadata`** on the `bench-size` column to combine runtime results with the relevant metadata.

- **Save the merged DataFrame** to a CSV file (`runtime_reported_runtime_discrepancies.csv`) for later analysis.  

In [5]:
# Merge benchmark_data with benchmark_metadata
comparison_df = benchmark_data.merge(
    benchmark_metadata,
    on="bench-size",
    how="left",
)

# Save merged data to CSV
output_csv = "../results/runtime_reported_runtime_discrepancies.csv"
comparison_df.to_csv(output_csv, index=False)

# Display the first few rows of `comparison_df`
display(comparison_df.head())

,bench-size,solver-version,Runtime (s),Reported Runtime (s),runtime-difference,runtime-difference-%,Num. variables,Num. constraints,bench-size-category
0,TIMES-GEO-global-netzero-31-20ts,gurobi-13.0.0,30621.769838,30590.708018,31.061819,0.101437,5422595,5063755,L
1,TIMES-GEO-global-netzero-31-20ts,highs-1.12.0,62869.744979,62820.879014,48.865966,0.077726,5422595,5063755,L
2,pypsa-de-elec-50-1h,gurobi-13.0.0,2989.140083,2923.996410,65.143673,2.179345,8637885,18370417,L
3,temoa-US_9R_TS-9-12ts,gurobi-13.0.0,190.656273,172.031106,18.625167,9.768977,1801794,1594079,L
4,temoa-US_9R_TS-9-12ts,highs-hipo-1.12.0-hipo,3181.154351,3181.154351,0.000000,0.000000,1801794,1594079,L


# 4. Identify Benchmark-Solver Pairs with Largest Runtime Discrepancies

In [6]:
# Sort the merged DataFrame by absolute runtime difference
comparison_df = comparison_df.sort_values(
    "runtime-difference-%",
    ascending=False,
)

# Save merged data to CSV
output_csv = "../results/runtime_reported_runtime_discrepancies.csv"
comparison_df.to_csv(output_csv, index=False)

# Display the top 20 rows to inspect the biggest mismatches
display(comparison_df.head(20))

,bench-size,solver-version,Runtime (s),Reported Runtime (s),runtime-difference,runtime-difference-%,Num. variables,Num. constraints,bench-size-category
1209,SWITCH-carbon-cap-3-6ts,gurobi-11.0.0,0.283666,0.002863,0.280803,98.990656,616,773,S
1221,SWITCH-carbon-cap-3-6ts,gurobi-12.0.0,0.246567,0.002840,0.243727,98.848165,616,773,S
1003,SWITCH-3-zone-tiny-3-6ts,gurobi-12.0.0,0.270315,0.003687,0.266628,98.636070,582,795,S
292,SWITCH-3zone-toy-stochastic-PySP-3-6ts,gurobi-12.0.0,0.227043,0.003222,0.223821,98.580890,580,716,S
431,SWITCH-hydrogen-3-6ts,gurobi-13.0.0,0.287482,0.004312,0.283170,98.500066,733,863,S
254,SWITCH-3zone-toy-stochastic-PySP-3-6ts,gurobi-10.0.0,0.201763,0.003182,0.198581,98.422931,580,716,S
1025,SWITCH-3-zone-tiny-3-6ts,gurobi-13.0.0,0.218158,0.003540,0.214618,98.377306,582,795,S
962,SWITCH-3-zone-tiny-3-6ts,gurobi-10.0.0,0.183444,0.003130,0.180314,98.293654,582,795,S
397,SWITCH-hydrogen-3-6ts,gurobi-12.0.0,0.263799,0.004502,0.259297,98.293373,733,863,S
1439,SWITCH-planning-reserves-3-6ts,gurobi-13.0.0,0.300677,0.005467,0.295210,98.181793,618,784,S


# 5. Extract Medium Runtime Differences

In [9]:
# Select the medium discrepancy cases (e.g., 40% to 50% of sorted rows)
medium = comparison_df.iloc[int(0.2 * len(comparison_df)) : int(0.3 * len(comparison_df))]

# Display the top 20 rows of medium discrepancies
display(medium.head(20))


,bench-size,solver-version,Runtime (s),Reported Runtime (s),runtime-difference,runtime-difference-%,Num. variables,Num. constraints,bench-size-category
200,Sienna_modified_RTS_GMLC_DA_sys_NetPTDF_Horizo...,gurobi-10.0.0,2.385977,2.133096,0.252881,10.598619,8820,10368,S
1241,FINE-1-node-energy-system-workflow-1-8760ts,highs-1.5.0.dev0,0.460686,0.412408,0.048277,10.479416,4892,8339,S
826,times-etimeseu-france-elec+heat-multi_stage-1-...,gurobi-10.0.0,5.978178,5.358131,0.620047,10.371843,92084,91288,M
942,OEMOF-v3-invest-optimize-only-storage-with-fos...,gurobi-13.0.0,6.120399,5.486202,0.634197,10.362017,61327,61329,M
1065,FINE-spatial-aggregation-of-energy-system-mode...,gurobi-12.0.0,3.597952,3.232676,0.365276,10.152333,8034,15898,S
906,OEMOF-v3-invest-optimize-only-storage-with-fos...,gurobi-11.0.0,6.572313,5.911365,0.660948,10.056554,61327,61329,M
1173,FINE-water-supply-system-12-8760ts,highs-1.9.0,0.355716,0.320483,0.035233,9.904846,4701,6461,S
1258,FINE-1-node-energy-system-workflow-1-8760ts,highs-1.9.0,0.520511,0.469136,0.051376,9.870268,4892,8339,S
1386,genx-3_three_zones_w_co2_capture-no_uc-3-1h,gurobi-10.0.0,12.231588,11.026207,1.205381,9.854655,103506,114590,M
924,OEMOF-v3-invest-optimize-only-storage-with-fos...,gurobi-12.0.0,6.125544,5.522174,0.603370,9.850064,61327,61329,M
